In [6]:
import pansat
from pansat.products.satellite.

SyntaxError: invalid syntax (1652838923.py, line 2)

In [1]:
from pyarts.workspace import Workspace, arts_agenda

In [72]:
ws = Workspace()
ws.execute_controlfile("general/general.arts")
ws.execute_controlfile("general/continua.arts")
ws.execute_controlfile("general/agendas.arts")
ws.execute_controlfile("general/planet_earth.arts")

In [2]:
from pyarts.xml import save

In [11]:
import numpy as np
p = np.logspace(0, np.log10(101300), 101)

In [12]:
save(p[::-1], "transmissivity/p_grid.xml")

In [13]:
%cat transmissivity/p_grid.xml

<?xml version="1.0"?>
<arts version="1" format="ascii">
<Vector nelem="101">
1.0130000e+05
9.0272060e+04
8.0444667e+04
7.1687125e+04
6.3882965e+04
5.6928399e+04
5.0730936e+04
4.5208155e+04
4.0286607e+04
3.5900839e+04
3.1992524e+04
2.8509684e+04
2.5406001e+04
2.2640198e+04
2.0175491e+04
1.7979103e+04
1.6021823e+04
1.4277621e+04
1.2723299e+04
1.1338188e+04
1.0103865e+04
9.0039166e+03
8.0237127e+03
7.1502178e+03
6.3718153e+03
5.6781529e+03
5.0600055e+03
4.5091522e+03
4.0182671e+03
3.5808218e+03
3.1909986e+03
2.8436132e+03
2.5340456e+03
2.2581788e+03
2.0123440e+03
1.7932719e+03
1.5980488e+03
1.4240786e+03
1.2690474e+03
1.1308936e+03
1.0077798e+03
8.9806873e+02
8.0030122e+02
7.1317709e+02
6.3553766e+02
5.6635038e+02
5.0469511e+02
4.4975190e+02
4.0079003e+02
3.5715836e+02
3.1827661e+02
2.8362769e+02
2.5275080e+02
2.2523529e+02
2.0071524e+02
1.7886454e+02
1.5939260e+02
1.4204046e+02
1.2657734e+02
1.1279760e+02
1.0051799e+02
8.9575179e+01
7.9823651e+01
7.1133716e+01
6.3389803e+01
5.6488924e+01

In [74]:
ws.Copy( ws.iy_main_agenda, ws.iy_main_agenda__Emission )
ws.Copy( ws.iy_space_agenda, ws.iy_space_agenda__CosmicBackground )
ws.Copy( ws.iy_surface_agenda, ws.iy_surface_agenda__UseSurfaceRtprop )
ws.Copy( ws.ppath_agenda, ws.ppath_agenda__FollowSensorLosPath )
ws.Copy( ws.ppath_step_agenda, ws.ppath_step_agenda__GeometricPath )
ws.Copy( ws.propmat_clearsky_agenda, ws.propmat_clearsky_agenda__LookUpTable )
ws.Copy( ws.iy_main_agenda, ws.iy_main_agenda__Emission )
ws.Copy( ws.iy_space_agenda, ws.iy_space_agenda__CosmicBackground )
ws.Copy( ws.iy_surface_agenda, ws.iy_surface_agenda__UseSurfaceRtprop )
ws.Copy( ws.ppath_agenda, ws.ppath_agenda__FollowSensorLosPath )
ws.Copy( ws.ppath_step_agenda, ws.ppath_step_agenda__GeometricPath )
ws.Copy( ws.propmat_clearsky_agenda, ws.propmat_clearsky_agenda__LookUpTable )

In [75]:
print(ws.AtmRawRead)

ARTS Workspace Method: AtmRawRead


In [76]:
ws.abs_speciesSet(
    species=[
        "H2O, H2O-SelfContCKDMT252, H2O-ForeignContCKDMT252",
        "CO2, CO2-CKDMT252",
        "O3",
        "CH4",
        "N2O"
    ]
)

# Absorption look-up table
#
ws.VectorSet( ws.abs_t_pert, [] )
ws.abs_speciesSet(abs_species=ws.abs_nls, species=[] )
ws.VectorSet( ws.abs_nls_pert, [] )
#
ws.IndexSet( ws.abs_p_interp_order,   3 )
ws.IndexSet( ws.abs_t_interp_order,   3 )
ws.IndexSet( ws.abs_nls_interp_order, 3 )


# Atmosphere, basics
#
ws.AtmosphereSet1D()
ws.AtmRawRead( basename = "/home/simonpf/src/arts-xml-data/planets/Earth/Fascod/tropical/tropical")


# Surface (if changed, also change in cfile_allsky for DISORT)
#
ws.MatrixSet( ws.z_surface, [0] )
@arts_agenda
def surface_rtprop_agenda(ws):
    ws.InterpAtmFieldToPosition( out=ws.surface_skin_t, field=ws.t_field)
    ws.surfaceBlackbody()
ws.surface_rtprop_agenda = surface_rtprop_agenda

# Frequency and wavenumber grids 




In [79]:
from scipy.constants import c
import numpy as np
lambdas = np.logspace(-7, 0, 256)[::-1]
f = c / lambdas

ws.f_grid = f

In [80]:
# 
ws.stokes_dim = 1
ws.sensor_pos = np.array([100e3])
ws.sensor_los = np.array([180])
ws.iy_unit = "PlanckBT"
# 
ws.sensorOff()
ws.jacobianOff()


In [81]:
ws.p_grid = np.logspace(np.log10(1013_00), np.log10(1), 41)
ws.AtmFieldsCalc()

In [82]:
ws.cloudboxOff()

In [83]:
ws.propmat_clearsky_agenda_checkedCalc()
ws.atmfields_checkedCalc()
ws.atmgeom_checkedCalc()
ws.cloudbox_checkedCalc()
ws.sensor_checkedCalc()

In [84]:
ws.Copy(ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA)

In [85]:
ws.abs_xsec_agenda_checkedCalc()

In [86]:
ws.abs_lookupSetup()
ws.abs_lookupCalc()


In [89]:
ws.yCalc()

In [94]:
ws.sensor_pos.value

array([[100000.]])

In [ ]:
%ls /home/simonpf/src/arts-xml-data/fa

In [ ]:
Arts2{

INCLUDE "general/general.arts"
INCLUDE "general/continua.arts"
INCLUDE "general/agendas.arts"
INCLUDE "general/planet_earth.arts"

# (standard) emission calculation
Copy( iy_main_agenda, iy_main_agenda__Emission )

# cosmic background radiation
Copy( iy_space_agenda, iy_space_agenda__CosmicBackground )

# standard surface agenda (i.e., make use of surface_rtprop_agenda)
Copy( iy_surface_agenda, iy_surface_agenda__UseSurfaceRtprop )

# sensor-only path
Copy( ppath_agenda, ppath_agenda__FollowSensorLosPath )

# no refraction
Copy( ppath_step_agenda, ppath_step_agenda__GeometricPath )

# absorption from LUT
Copy( propmat_clearsky_agenda, propmat_clearsky_agenda__LookUpTable )


# Read parameters
#
StringCreate( atm )
StringCreate( tablename )
NumericCreate( wn_start )
NumericCreate( wn_end )
NumericCreate( dwn )
#
ReadXML( atm, "atm.xml" )
ReadXML( tablename, "tablename.xml" )
ReadXML( wn_start, "wn_start.xml" )
ReadXML( wn_end, "wn_end.xml" )
ReadXML( dwn, "dwn.xml" )



ARTS Workspace Variable: y